In [181]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *
from dataPrep import *
from sklearn.metrics import *
from scipy.stats import linregress

from sklearn.metrics.pairwise import *

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [156]:
%%time
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])

##Long dataframes
df2_A = pd.read_csv('data/prodIng_Wide_01.csv', index_col=0)
df2_B = pd.read_csv('data/prodIng_Wide_01.csv', index_col=0)
df2_C = pd.read_csv('data/prodIng_Wide_01.csv', index_col=0)
frames = [df2_A, df2_B, df2_C]
df3 = pd.concat(frames)

print('Number of products df: ', len(prods))
print('Number of products: df2', len(df3))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products df:  15945
Number of products: df2 15945
Number of unique ingredients:  6899
Number of total ingredients:  446618


In [157]:
##create function for removing water (or other ingredients) from ingredient listings
def  remove_values_from_list (the_list, val):
   return [value for value in the_list if value != val]

df = dataPrep(ings, prods, prod_ing)
#df.to_csv('recommendations/recommender_products.csv')

In [184]:
df2 = df2.T

In [4]:
#df = df[df['ingCount'] > 1]  
#df_cos = df
#print('Number of products: ', len(df))

# GET RECOMMENDATIONS


In [191]:
prodid =  1778    
product ='Glow Tonic'

##get indices of all products
indices = pd.Series(df.index, index=df['product']).drop_duplicates()
##find index of specified product
idx = indices[product]

print('Product:', df.iloc[prodid]['product'])
print('Number of ingredients:', df.iloc[prodid]['ingCount'])
print('Ingredients', df.iloc[prodid]['ingList'])

Product: Glow Tonic
Number of ingredients: 24
Ingredients ['Water', 'Aloe Barbadensis Leaf Juice', 'Hamamelis Virginiana (Witch Hazel) Leaf Extract', 'Aesculus Hippocastanum (Horse Chestnut) Seed Extract', 'Glycolic Acid', 'Ammonium Glycolate', 'Glycerin', 'Butylene Glycol', 'Hexylene Glycol', 'Fructose', 'Glucose', 'Sucrose', 'Urea', 'Dextrin', 'Alanine', 'Glutamic Acid', 'Aspartic Acid', 'Hexyl Nicotinate', 'Panax Ginseng Root Extract', 'Phenoxyethanol', 'Sodium Benzoate', 'Biotin', 'Polysorbate 20', 'Fragrance']


### Get other ingredient list

In [107]:
prod_ing_grab = 14768

print('Product:', df.iloc[prod_ing_grab]['product'])
print('Number of ingredients:', df.iloc[prod_ing_grab]['ingCount'])
print('Ingredients', df.iloc[prod_ing_grab]['ingList'])

Product: Glow Tonic To-Go Pads
Number of ingredients: 29
Ingredients ['Water', 'Glycolic Acid', 'Butylene Glycol', 'Glycerin', 'Sodium Hydroxide', 'Aloe Barbadensis Leaf Juice', 'Hamamelis Virginiana (Witch Hazel) Water', 'Aesculus Hippocastanum (Horse Chestnut) Seed Extract', 'Hexylene Glycol', 'Fructose', 'Glucose', 'Sucrose', 'Urea', 'Dextrin', 'Alanine', 'Glutamic Acid', 'Aspartic Acid', 'Hexyl Nicotinate', 'Panax Ginseng Root Extract', 'Ethylhexylglycerin', 'Disodium EDTA', 'Biotin', 'Panthenol', 'PPG-26-Buteth-26', 'PEG-40 Hydrogenated Castor Oil(Perfuming, Surfactant, Fragrance, Emulsifying, ', 'Phenoxyethanol', 'Fragrance', 'Caramel', 'CI 14700']


## Cosine similarity

Reference [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

### Create dataframe of top ten similar products

In [64]:
%%time
column = 'product'
# Get the index of the product that matches the product name
df['ing#List'] = df['ing#List'].astype(str)
tfidf = TfidfVectorizer(binary = True, stop_words = None)
tfidf_matrix = tfidf.fit_transform(df['ing#List'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df[column]).drop_duplicates()
idx = indices[product]

# Get the pairwsie similarity scores of all products with that product
sim_scores_all = list(enumerate(cosine_sim[idx]))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

# Return the top 10 most similar products and their ingredients
#return df[['product', 'brand','ingList']].iloc[product_indices]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 51.6 s


### Show top ten

In [108]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/COS.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,3743,Geranium Face Toner,TheBalm,"[Water, Aloe Barbadensis Leaf Juice, Glycolic ...",8
1,14011,Resilient Stem Cell Regenerating Treatment,Dermelect,"[Water, Aloe Barbadensis Leaf Juice, Cyclopent...",6
2,225,Acne Dote Face & Body Scrub,Alba Botanica,"[Water, Aloe Barbadensis Leaf Juice, Sodium La...",4
3,9960,Ultra-Light Oil Control Lotion,Organic Surge,"[Water, Aloe Barbadensis Leaf Juice, Argania S...",3
4,9958,Radiance Recovery Night Cream,Organic Surge,"[Water, Aloe Barbadensis Leaf Juice, Argania S...",3
5,11876,Jelly Sunscreen SPF50 PA+++,Look At Me,"[Water, Aloe Barbadensis Leaf Juice, Orchid Ex...",6
6,9956,Nourishing Rose Day Cream,Organic Surge,"[Water, Aloe Barbadensis Leaf Juice, Argania S...",3
7,9689,Sea Algae Serum,Evanhealy,"[Water, Aloe Barbadensis Leaf Juice, Lavandula...",3
8,13739,Organic Surge Daily Care Refreshing Face Wash,Organic Surge,"[Water, Aloe Barbadensis Leaf Juice, Sodium Co...",4
9,3363,Moroccan Neroli Shaving Serum,Aesop,"[Water, Aloe Barbadensis Leaf Juice, Sodium La...",4


### Find specific product similarity

In [73]:
prod_search_id = 14768
[v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

[(14768, 0.757428326538646)]

## RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

### Create dataframe of top ten similar products

In [159]:
%%time

##Set the rbo we want to use
rbo = rbo_min

#df['ing#List'] = [ast.literal_eval(i) for i in df['ing#List']]
indices = pd.Series(df.index, index=df['product']).drop_duplicates()
idx = indices[product]

itemLookup = df.loc[idx]['ing#List']
items = df['ing#List']
sim = [rbo(itemLookup,i, .9) for i in items] ##change method here

sim_scores_all = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 3.59 s


### Show top ten

In [160]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/RBO.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,14768,Glow Tonic To-Go Pads,Pixi,"[Water, Glycolic Acid, Butylene Glycol, Glycer...",20
1,3743,Geranium Face Toner,TheBalm,"[Water, Aloe Barbadensis Leaf Juice, Glycolic ...",8
2,3707,Alcohol-Free Face Toner,TheBalm,"[Water, Hamamelis Virginiana (Witch Hazel) Lea...",9
3,8285,Clear Skin Hydrator Gel,Alpha-H,"[Water, Hamamelis Virginiana (Witch Hazel) Lea...",6
4,694,Seaweed Cleansing Lotion,Mario Badescu,"[Water, Aloe Barbadensis Leaf Juice, Propylene...",7
5,7541,Sensitive Facial Cleansing Towelettes with Cot...,Burt's Bees,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",8
6,1372,Facial Radiance Pads,First Aid Beauty,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",5
7,11875,Orchid Flower Aqua UV Sun Gel,The Orchid Skin,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",6
8,10629,Green Cover Sunscreen SPF50+ PA+++,Cosmo Monster,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",5
9,6271,Sensitive Facial Cleansing Towelettes,Burt's Bees,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",8


### Find specific product similarity

In [161]:
prod_search_id = 14768
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims)
print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
print('Ingredients', df.iloc[prod_search_id]['ingList'])

Product: Glow Tonic To-Go Pads
Similarity score: [(14768, 0.6488925800314376)]
Number of Ingredients 29
Ingredients ['Water', 'Glycolic Acid', 'Butylene Glycol', 'Glycerin', 'Sodium Hydroxide', 'Aloe Barbadensis Leaf Juice', 'Hamamelis Virginiana (Witch Hazel) Water', 'Aesculus Hippocastanum (Horse Chestnut) Seed Extract', 'Hexylene Glycol', 'Fructose', 'Glucose', 'Sucrose', 'Urea', 'Dextrin', 'Alanine', 'Glutamic Acid', 'Aspartic Acid', 'Hexyl Nicotinate', 'Panax Ginseng Root Extract', 'Ethylhexylglycerin', 'Disodium EDTA', 'Biotin', 'Panthenol', 'PPG-26-Buteth-26', 'PEG-40 Hydrogenated Castor Oil(Perfuming, Surfactant, Fragrance, Emulsifying, ', 'Phenoxyethanol', 'Fragrance', 'Caramel', 'CI 14700']


In [162]:
#list1 = df.iloc[prod_search_id]['ing#List']
#list2 = df.iloc[1778]['ing#List']
#rbo(list1,list2, .9)

## R value
Reference [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.linregress.html)

In [185]:
%%time

indices = pd.Series(df.index, index=df['product']).drop_duplicates()
idx = indices[product]
sim = [linregress(df2[idx], df2[i])[2] for i in df2]

sim_scores = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#df['scores'] = [sim_scores[i][1] for i in sim_scores]

# Get the scores of the 10 most similar products
sim_scores = sim_scores[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 44.3 s


### Return top ten

In [186]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/RBO.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,8032,Revitalising Ginseng Toner With Glycolic Acid 5%,Bravura London,"[Water, Aloe Barbadensis (Aloe Vera) Gel, Hama...",17
1,14768,Glow Tonic To-Go Pads,Pixi,"[Water, Glycolic Acid, Butylene Glycol, Glycer...",20
2,13600,Tensage Eye Contour Cream,Biopelle,"[Water, Snail Secretion Filtrate, C12-20 Acid ...",14
3,10534,Rehydrating Toning Essence,Su-Man Skincare,"[Water, Rosa Centifolia Flower Water, Hyaluron...",13
4,812,Glycolic Acid 7% Toning Solution,The Ordinary,"[Water, Glycolic Acid, Rosa Damascena Flower W...",16
5,2962,Cellular 3-Minute Peel,La Prairie,"[Water, Hydroxyethyl Acrylate/Sodium Acryloyld...",15
6,3350,Juniper Oil-Free Face Moisturizer,TheBalm,"[Water, Ethylhexyl Methoxycinnamate, Butylene ...",14
7,1760,Skin Hydrating Masque,Dermalogica,"[Water, Glycerin, Butylene Glycol, Squalane, D...",14
8,9098,Tensage Radiance Eye Cream,Biobelle,"[Water, Snail Secretion Filtrate, C12-20 Acid ...",14
9,2866,Ultra Facial Oil-Free Lotion,Kiehl's,"[Water, Glycerin, Alcohol Denat., Isononyl Iso...",13


### Find specific product similarity

In [192]:
prod_search_id = 14768
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims)
print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
print('Ingredients', df.iloc[prod_search_id]['ingList'])

Product: Glow Tonic To-Go Pads
Similarity score: [(14768, 0.6488925800314376)]
Number of Ingredients 29
Ingredients ['Water', 'Glycolic Acid', 'Butylene Glycol', 'Glycerin', 'Sodium Hydroxide', 'Aloe Barbadensis Leaf Juice', 'Hamamelis Virginiana (Witch Hazel) Water', 'Aesculus Hippocastanum (Horse Chestnut) Seed Extract', 'Hexylene Glycol', 'Fructose', 'Glucose', 'Sucrose', 'Urea', 'Dextrin', 'Alanine', 'Glutamic Acid', 'Aspartic Acid', 'Hexyl Nicotinate', 'Panax Ginseng Root Extract', 'Ethylhexylglycerin', 'Disodium EDTA', 'Biotin', 'Panthenol', 'PPG-26-Buteth-26', 'PEG-40 Hydrogenated Castor Oil(Perfuming, Surfactant, Fragrance, Emulsifying, ', 'Phenoxyethanol', 'Fragrance', 'Caramel', 'CI 14700']
